In [1]:
import numpy as np
from pathlib import Path
import json 
from PIL import Image
from random import randint
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import math
import torch.nn.functional as F
import copy
from monai.losses import DiceCELoss
from monai.networks.nets import UNETR
from monai.transforms import (
    AsDiscrete,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
)
from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)
from tqdm import tqdm
import os
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from sklearn.model_selection import train_test_split
import pickle

In [7]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


class DeviceDataloader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    def __len__(self):
        return len(self.dl)
    

class DataTrain(Dataset):

    def __init__(self, data, annotation):
        self.traininputtensor = data
        self.output = annotation
    
    def __getitem__(self, index):
        input_image = self.traininputtensor[index] 
        output_label = self.output[index]  
        return input_image, output_label

    def __len__(self):
        return self.traininputtensor.size(0)


class DataTest(Dataset):

    def __init__(self, data, annotation):
        self.testinputtensor = torch.tensor(data, dtype=torch.float).unsqueeze(0)
        self.output = torch.tensor(annotation, dtype=torch.float)

    def __getitem__(self, index):
        input_image = self.testinputtensor[index]
        output_label = self.output[index] 
        return input_image, output_label

    def __len__(self):
        return self.testinputtensor.size(0)


def reshape_mask(mask, num_clases=8, depth=272):
    """We wanted to convert our 3d masks that have shape (1,D,H,W) to the shape (num_classes,D,H,W) """
    new_mask = np.zeros((num_clases, depth, 512, 512), dtype=np.uint8)

    for z in range(depth):  #iteration over 'z' axis 
        for y in range(512):    #iteration over 'y' axis
            for x in range(512):    #iteration over 'x' axis
                value = mask[z, x, y]
                if value!=0:
                    trida = int(value/25)
                    if trida==10:
                        trida=8
                    new_mask[trida - 1, z, x, y] = 1

    print("New mask shape: ", new_mask.shape)

    return new_mask



def cut_data(data, z_shape=272):
    """Metacentrum haven't got enough memory to process one full image, that's why it will be cutted """
    cutted_data = []

    for i in range(0, data.shape[0], z_shape):
        new_data = np.array(data[i:i+z_shape, :, :])
        cutted_data.append(new_data)

    #for neural network we must have same dimension
    if cutted_data[-1].shape[0] < z_shape:
        cutted_data.pop(-1)

    return cutted_data

In [3]:
masks_path = Path(r'C:\working_space\pilsen_pigs_2023_cvat_backup\masks_for_all_organs')
data_path = Path(r'C:\working_space\pilsen_pigs_2023_cvat_backup\workspase')

sorted_pictures = sorted(
        [d.name for d in data_path.iterdir() if d.is_dir() and d.name.isdigit()],
        key=lambda x: int(x)
    )

sorted_masks = sorted(
        [d.name for d in masks_path.iterdir() if d.is_dir() and d.name.isdigit()],
        key=lambda x: int(x)
    )



images_3d_full = []
for image_3d in sorted_pictures:
    images_path = Path(fr'C:\working_space\pilsen_pigs_2023_cvat_backup\workspase\{image_3d}\data')
    images = []
    for image in images_path.iterdir():
        img = Image.open(image)
        img = img.convert('L')
        img = np.array(img)
        images.append(img)
    images_3d_full.append(np.array(images))
print("Shape of the first 3D image: ", images_3d_full[0].shape)



masks_3d_full = []
for mask_3d in sorted_masks:
    masks_path = Path(fr'C:\working_space\pilsen_pigs_2023_cvat_backup\masks_for_all_organs\{mask_3d}')
    masks = []
    for mask in masks_path.iterdir():
        img = Image.open(mask)
        img = np.array(img)
        masks.append(img)
    masks_3d_full.append(np.array(masks))
print("Shape of the first 3D mask", masks_3d_full[0].shape)

Shape of the first 3D image:  (882, 512, 512)
Shape of the first 3D mask (882, 512, 512)


In [4]:
trainX = []
trainY = []


cutted_5 = cut_data(images_3d_full[4])
cutted_14 = cut_data(images_3d_full[13])

cutted_mask_5 = cut_data(masks_3d_full[4])
cutted_mask_14 = cut_data(masks_3d_full[13])

for i in range(len(cutted_5)):
    trainX.append(torch.tensor(cutted_5[i]).unsqueeze(0))
    trainY.append(reshape_mask(cutted_mask_5[i]))

for i in range(len(cutted_14)):
    trainX.append(torch.tensor(cutted_14[i]).unsqueeze(0))
    trainY.append(reshape_mask(cutted_mask_14[i]))

print(trainX[0].shape)

New mask shape:  (8, 272, 512, 512)
New mask shape:  (8, 272, 512, 512)
New mask shape:  (8, 272, 512, 512)
New mask shape:  (8, 272, 512, 512)
New mask shape:  (8, 272, 512, 512)
New mask shape:  (8, 272, 512, 512)
New mask shape:  (8, 272, 512, 512)
torch.Size([1, 272, 512, 512])


In [9]:
data = DataTrain(trainX, trainY)
dataloader_train = DataLoader(dataset=torch.tensor(data, dtype=torch.float), batch_size=1, shuffle=True)
for x, y in dataloader_train:
    print("Shape of x", x.shape)
    print("Shape of y", y.shape)

AttributeError: 'list' object has no attribute 'size'

In [ ]:
trainX = []
trainY = []


cutted_5 = cut_data(images_3d_full[4])
cutted_14 = cut_data(images_3d_full[13])

cutted_mask_5 = cut_data(masks_3d_full[4])
cutted_mask_14 = cut_data(masks_3d_full[13])

for i in range(len(cutted_5)):
    trainX.append(cutted_5[i])
    trainY.append(reshape_mask(cutted_mask_5[i]))

for i in range(len(cutted_14)):
    trainX.append(cutted_14[i])
    trainY.append(reshape_mask(cutted_mask_14[i]))


X_for_mix = []
Y_for_mix = []
for i in range(len(images_3d_full)):

    if i != 4 and i != 13:

        cutted = cut_data(images_3d_full[i])
        cutted_mask = cut_data(masks_3d_full[i])

        X_for_mix.extend(cutted)
        Y_for_mix.extend(cutted_mask)


res_train_x, testX, res_train_y, testY = train_test_split(X_for_mix, Y_for_mix, test_size=0.33)
trainX.extend(res_train_x)
trainY.extend(res_train_y)


data_train = DataTrain(np.array(trainX), np.array(trainY))
dataloader_train = DataLoader(dataset=data_train, batch_size=1, shuffle=True)

data_test = DataTrain(np.array(testX), np.array(testY))
dataloader_test = DataLoader(dataset=data_test, batch_size=1, shuffle=True)

New mask shape:  (8, 272, 512, 512)
New mask shape:  (8, 272, 512, 512)
New mask shape:  (8, 272, 512, 512)
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_19352\3340582374.py", line 17, in <module>
    trainY.append(reshape_mask(cutted_mask_14[i]))
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Local\Temp\ipykernel_19352\1574406686.py", line None, in reshape_mask
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^